# In silico identification of switching nodes (ISIS), by F. Mairet
## <I>Saccharomyces cerevisiae</I> under nitrogen limitation

In [1]:
import cobra
import numpy as np
from pandas import DataFrame
import pandas as pd
model=cobra.io.read_sbml_model("yeastGEM.xml")

### Computation of fluxes with pFBA

In [2]:
medium = model.medium
medium["r_1992"] = 10.0 #oxygen
medium["r_1714"] = 10.0 #glucose
medium["r_1654"] = 1.0 #NH4
model.medium = medium
pfba_NH4 = cobra.flux_analysis.pfba(model)

medium = model.medium
medium["r_1992"] = 10.0 #oxygen
medium["r_1714"] = 10.0 #glucose
medium["r_1654"] = 0 #Nh4
medium["r_1873"] = 1 #alanine
model.medium = medium
pfba_Ala = cobra.flux_analysis.pfba(model)

medium = model.medium
medium["r_1992"] = 10.0 #oxygen
medium["r_1714"] = 10.0 #glucose
medium["r_1654"] = 0 #Nh4
medium["r_1873"] = 0 #alanine
medium["r_1891"] = 1 #glutamine
model.medium = medium
pfba_Gln = cobra.flux_analysis.pfba(model)

# Concatenation and normalization of fluxes
VV=(np.concatenate((np.array(pfba_NH4.fluxes)[np.newaxis,:],np.array(pfba_Ala.fluxes)[np.newaxis,:],np.array(pfba_Gln.fluxes)[np.newaxis,:]),axis=0)).T
VV=np.divide(VV,np.linalg.norm(VV,axis=0))

df1=DataFrame(VV,index=model.reactions,columns=['NH4','Ala','Gln'])
df1.to_excel('Scerevisiae_Flux.xlsx', sheet_name='sheet1', index=True)

### Identification of switch nodes

In [3]:
S=cobra.util.array.create_stoichiometric_matrix(model,array_type='dense') # stoichiometric matrix
nm=S.shape[0] # number of metabolites

#### NH4 vs Ala

In [19]:
V=VV[:,[0,1]]

# loop on all the metabolites to compute their score
r=np.zeros(nm)
for i in range (0,nm):
    jj=S[i,:]!=0
    M=np.multiply(S[i,jj][np.newaxis,:].T, V[jj,:]) # compute M, removing the zero rows 
    
    u, s, vh = np.linalg.svd(M)
    if s[0]==0:
        r[i]=0
    else:
        r[i]=1-pow(s[0],2)/sum(pow(s,2))
        
# Export and print the results
df2=pd.concat([DataFrame(model.metabolites, columns=['Metabolite']), DataFrame(r, columns=['score'])], axis = 1) 

ind=np.argsort(r)
for i in range (1,20):
   print(i,model.metabolites[ind[-i]].name,':',r[ind[-i]])


1 D-xylulose 5-phosphate [cytoplasm] : 0.2576631370017013
2 L-tyrosine [cytoplasm] : 0.24997082735711307
3 L-phenylalanine [cytoplasm] : 0.24997082735711307
4 keto-phenylpyruvate [cytoplasm] : 0.24997082735711307
5 3-(4-hydroxyphenyl)pyruvate [cytoplasm] : 0.24997082735711296
6 phosphatidylethanolamine (1-16:0, 2-18:1) [endoplasmic reticulum membrane] : 0.24722314346365093
7 phosphatidyl-L-serine (1-16:0, 2-18:1) [endoplasmic reticulum membrane] : 0.24722314346364704
8 citrate [cytoplasm] : 0.24007907068218903
9 stearate [mitochondrial membrane] : 0.21972357959601463
10 L-glutamate [cytoplasm] : 0.17431338476287428
11 2-oxoglutarate [cytoplasm] : 0.17430904369805145
12 monolysocardiolipin (2-16:1, 3-18:0, 4-16:1) [mitochondrial membrane] : 0.14809341967675504
13 H+ [extracellular] : 0.13069269395964078
14 D-erythrose 4-phosphate [cytoplasm] : 0.12297577895601852
15 D-ribulose 5-phosphate [cytoplasm] : 0.10446080223853749
16 cardiolipin (1-18:1, 2-16:1, 3-18:0, 4-16:1) [mitochondrial me

#### NH4 vs Gln

In [20]:
V=VV[:,[0,2]]

# loop on all the metabolites to compute their score
r=np.zeros(nm)
for i in range (0,nm):
    jj=S[i,:]!=0
    M=np.multiply(S[i,jj][np.newaxis,:].T, V[jj,:]) # compute M, removing the zero rows 
    
    u, s, vh = np.linalg.svd(M)
    if s[0]==0:
        r[i]=0
    else:
        r[i]=1-pow(s[0],2)/sum(pow(s,2))
        
# Export and print the results
df3=pd.concat([DataFrame(model.metabolites, columns=['Metabolite']), DataFrame(r, columns=['score'])], axis = 1) 

ind=np.argsort(r)
for i in range (1,20):
   print(i,model.metabolites[ind[-i]].name,':',r[ind[-i]])



1 L-aspartate [cytoplasm] : 0.3574613910298361
2 D-erythrose 4-phosphate [cytoplasm] : 0.3513451903539815
3 H+ [extracellular] : 0.31595811940619767
4 L-glutamic 5-semialdehyde [cytoplasm] : 0.24487353740465123
5 L-isoleucine [cytoplasm] : 0.24487353740465123
6 (S)-3-methyl-2-oxopentanoate [mitochondrion] : 0.24487353740465123
7 7-phospho-2-dehydro-3-deoxy-D-arabino-heptonic acid [cytoplasm] : 0.24487353740465112
8 succinate [cytoplasm] : 0.24361428433173216
9 NADP(+) [mitochondrion] : 0.23864729470217994
10 NADPH [mitochondrion] : 0.23864729470217994
11 dADP [cytoplasm] : 0.22943310149637808
12 D-xylulose 5-phosphate [cytoplasm] : 0.2121875130523726
13 (S)-malate [cytoplasm] : 0.14096234022419873
14 D-ribulose 5-phosphate [cytoplasm] : 0.11988478333815367
15 L-glutamate [cytoplasm] : 0.10222893747999973
16 2-oxoglutarate [mitochondrion] : 0.08279255202208435
17 L-glutamine [cytoplasm] : 0.08276240587472883
18 isocitrate [mitochondrion] : 0.07471289678410442
19 coenzyme A [mitochondrio

#### Ala vs Gln

In [23]:
V=VV[:,[1,2]]

# loop on all the metabolites to compute their score
r=np.zeros(nm)
for i in range (0,nm):
    jj=S[i,:]!=0
    M=np.multiply(S[i,jj][np.newaxis,:].T, V[jj,:]) # compute M, removing the zero rows 
    
    u, s, vh = np.linalg.svd(M)
    if s[0]==0:
        r[i]=0
    else:
        r[i]=1-pow(s[0],2)/sum(pow(s,2))
        
# Export and print the results
df4=pd.concat([DataFrame(model.metabolites, columns=['Metabolite']), DataFrame(r, columns=['score'])], axis = 1) 

with pd.ExcelWriter("Scerevisiae_Metabolite.xlsx") as writer:
    df2.to_excel(writer, sheet_name="NH4_vs_Ala", index=False)
    df3.to_excel(writer, sheet_name="NH4_vs_Gln", index=False)
    df4.to_excel(writer, sheet_name="Ala_vs_Gln", index=False)
        
ind=np.argsort(r)
for i in range (1,25):
   print(i,model.metabolites[ind[-i]].name,':',r[ind[-i]])


1 L-glutamate [cytoplasm] : 0.38186450809809525
2 L-aspartate [cytoplasm] : 0.3573038197639541
3 2-oxoglutarate [cytoplasm] : 0.34533519608066787
4 L-tyrosine [cytoplasm] : 0.24409647674582635
5 L-phenylalanine [cytoplasm] : 0.24409647674582635
6 keto-phenylpyruvate [cytoplasm] : 0.24409647674582635
7 L-glutamic 5-semialdehyde [cytoplasm] : 0.24409647674582624
8 3-(4-hydroxyphenyl)pyruvate [cytoplasm] : 0.24409647674582624
9 (S)-3-methyl-2-oxopentanoate [mitochondrion] : 0.24409647674582624
10 L-isoleucine [cytoplasm] : 0.24409647674582624
11 7-phospho-2-dehydro-3-deoxy-D-arabino-heptonic acid [cytoplasm] : 0.2440964767458258
12 D-erythrose 4-phosphate [cytoplasm] : 0.24409647674582557
13 succinate [cytoplasm] : 0.24284371829473184
14 NADP(+) [mitochondrion] : 0.23738474999907655
15 NADPH [mitochondrion] : 0.23738474999907655
16 dADP [cytoplasm] : 0.2286276015877977
17 phosphatidylethanolamine (1-16:0, 2-18:1) [endoplasmic reticulum membrane] : 0.2237898281348587
18 phosphatidyl-L-seri